<a href="https://colab.research.google.com/github/AbdelnasserMostafa/-myproject/blob/master/Self_Organizing_Map_SOM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clustering using a Self-Organizing Map
# Setting up the SOM (Self-Organizing Map) algorithm

import tensorflow as tf
import numpy as np

class SOM:
  def __init__(self, width, height, dim):
    self.num_iters = 100
    self.width = width
    self.height = height
    self.dim = dim
    self.node_locs = self.get_locs()
    
    # Each node is a vector of dimension 'dim'. For a 2D grid, there are 'width * hight' nodes
    nodes = tf.Variable(tf.random_normal([width*height, dim]))
    self.nodes = nodes
    
    # Thes two ops are inputs at each iteration
    x = tf.placeholder(tf.floar32, [dim])
    iter = tf.placeholder(tf.float32)
    
    # We'll need to access them from another method
    self.x = x
    self.iter = iter
    
    # Find the node that matches closest to the input
    bmu_loc = self.get_bmu_loc(x)
    
    # Update the values of the neighbors
    self.propagate_nodes = self.get_propagation(bmu_loc, x, iter)
    
# Defining how to update the values of neighbors

def get_propagation(self, bmu_loc, x, iter):
  num_nodes = self.width * self.height
  
  # The rate decreases as iter increases. This value influences alpha and sigma parameters.
  rate = 1.0 - tf.div(iter, self.num_iters)
  alpha = rate * 0.5
  sigma = rate * tf.to_float(tf.maximum(self.width, self.height)) / 2.
  
  # Expand bmu_loc, so we can efficiently compare it pairwise with each element of note_locs
  expanded_bmu_loc = tf.expand_dims(tf.to_float(bmu_loc), 0)
  sqr_dists_from_bmu = tf.reduce_sum(
    tf.square(tf.subtract(expanded_bmu_loc, self.node_locs)), 1)
  
  # Ensure that nodes closer to the bmu charge more dramatically
  neigh_factor = tf.exp(-tf.div(sqr_dists_from_bmu, 2 * tf.square(sigma)))
  rate = tf.multiply(alpha, neigh_factor)
  rate_factor = tf.stack([tf.tile(tf.slice(rate, [i], [i]),
                                 [self.dim]) for i in range(num_nodes)])
  nodes_diff = tf.multiply(rate_factor,
                          tf.subtract(tf.stack([x for i in range(num_nodes)]), self.nodes))
  
  # Define the updates
  update_nodes = tf.add(self.nodes, nodes_diff)
  
  # Return an op to perform the updates
  return tf.assign(self.nodes, update_nodes)

# Get the node location of the closest match

def get_bmu_loc(self, x):
  expanded_x = tf.expand_dims(x, 0)
  sqr_diff = tf.square(tf.subtract(expanded_x, self.nodes))
  dists = tf.reduce_sum(sqr_diff, 1)
  bmu_idx = tf.argmin(dists, 0)
  bmu_loc = tf.stack([tf.mod(bmu_idx, self.width), tf.div(bmu_idx, self.width)])
  return bmu_loc

# Generate a matrix of points

def get_locs(self):
  locs = [[x, y]
         for y in range(self.height)
         for x in range(self.width)]
  return tf.to_float(locs)

# Run the SOM (Self-Organized MAP) algorithm

def train(self, data):
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(self.num_iters):
      for data_x in data:
        sess.run(self.propagate_nodes, feed_dict={self.x: data_x, self.iter: i})
    centroid_grid = [[] for i in range(self.width)]
    self.nodes_val = list(sess.run(self.nodes))
    self.locs_val = list(sess.run(self.node_locs))
    for i, l in enumerate(self.locs_val):
      centroid_grid[int(1[0])].append(self.nodes_val[i])
    self.centroid_grid = centroid_grid
    
# Test out and visualize results

from matplotlib import pyplot as plt
import numpy as np
from som import SOM

colors = np.array( 
    [[0., 0., 1.],
     [0., 0., 0.95],
     [0., 0.05, 1.], 
     [0., 1., 0.], 
     [0., 0.95, 0.], 
     [0., 1, 0.05], 
     [1., 0., 0.], 
     [1., 0.05, 0.], 
     [1., 0., 0.05], 
     [1., 1., 0.]])

# The grid size is 4 x 4, and the Input dimension is 3
som = SOM(4, 4, 3)
som.train(colors)

plt.imshow(som.centroid_grid)
plt.show()